In [2]:
# import modules
import pandas as pd

In [4]:
# declare files to load
school_data_to_load = 'Resources/schools_complete.csv'
student_data_to_load = 'Resources/students_complete.csv'

In [5]:
# read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load, encoding='utf-8')
student_data = pd.read_csv(student_data_to_load, encoding='utf-8')

In [6]:
# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [7]:
# display shape of schools dataframe
school_data_complete.shape

(39170, 11)

In [8]:
# list columns of schools dataframe
list(school_data_complete.columns)

['Student ID',
 'student_name',
 'gender',
 'grade',
 'school_name',
 'reading_score',
 'math_score',
 'School ID',
 'type',
 'size',
 'budget']

In [9]:
# District Summary

# total schools
list_of_schools = school_data_complete['school_name'].unique()
districtSchools = len(list_of_schools)

# total students
districtStudents = len(school_data_complete)

# total budget
districtBudget = school_data_complete[['school_name','budget']].drop_duplicates()['budget'].sum()

# average math score
districtAvgMath = school_data_complete['math_score'].mean()

# average reading score
districtAvgRead = school_data_complete['reading_score'].mean()

In [11]:
# percent passing math
passingScore = 70
cond = (school_data_complete['math_score'] >= passingScore)
districtPctPassMath = len(school_data_complete[cond])/districtStudents * 100

# percent passing reading
cond = (school_data_complete['reading_score'] >= passingScore)
districtPctPassRead = len(school_data_complete[cond])/districtStudents * 100

# percent overall passing rate
districtPctPassRate = (districtPctPassMath+districtPctPassRead)/2 * 100


In [12]:
print('    District Summary')
print('-----------------------')
print(f'Total Schools: {districtSchools}')
print(f'Total Students: {districtStudents}')
print(f'Total Budget: {districtBudget}')
print(f'Average Math Score: {districtAvgMath}')
print(f'Average Reading Score: {districtAvgRead}')
print(f'% Passing Math: {districtPctPassMath}')
print(f'% Passing Reading: {districtPctPassRead}')
print(f'% Overall Passing Rate: { districtPctPassRate}')
print()

District Summary
-----------------------
Total Schools: 15
Total Students: 39170
Total Budget: 24649428
Average Math Score: 78.98537145774827
Average Reading Score: 81.87784018381414
% Passing Math: 74.9808526933878
% Passing Reading: 85.80546336482001
% Overall Passing Rate: 8039.3158029103915



In [13]:
# prepare `District Summary` data

labels = [
    'Total Schools',
    'Total Students',
    'Total Budget',
    'Average Math Score',
    'Average Reading Score',
    '% Passing Math',
    '% Passing Reading',
    '% Overall Passing Rate'
]

summary = [[
    districtSchools,
    districtStudents,
    districtBudget,
    districtAvgMath,
    districtAvgRead,
    districtPctPassMath*100,
    districtPctPassRead*100,
    (districtPctPassMath+districtPctPassRead)/2 * 100
]]


## District Summary

In [14]:
district = pd.DataFrame(summary, columns=labels)
district['Total Students'] = district['Total Students'].map('{:,}'.format)
district['Total Budget'] = district['Total Budget'].map('${:,.2f}'.format)
district.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,7498.085269,8580.546336,8039.315803


In [15]:
# prepare schools summary data

labels = [
    'School Type',
    'Total Students',
    'Total School Budget',
    'Per Student Budget',
    'Average Math Score',
    'Average Reading Score',
    '% Passing Math',
    '% Passing Reading',
    '% Overall Passing Rate'
]

schools_summary = pd.DataFrame(index=list_of_schools)
schools_summary['School Type'] = school_data_complete.groupby('school_name')['type'].first()

schools_summary['Total Students'] = school_data_complete.groupby('school_name')['student_name'].count()

schools_summary['Total School Budget'] = school_data_complete[['school_name','budget']].drop_duplicates().groupby('school_name')['budget'].sum()

schools_summary['Per Student Budget'] = school_data_complete[['school_name','budget']].drop_duplicates().groupby('school_name')['budget'].sum() / school_data_complete.groupby('school_name')['student_name'].count()

schools_summary['Average Math Score'] = school_data_complete.groupby('school_name')['math_score'].mean()

schools_summary['Average Reading Score'] = school_data_complete.groupby('school_name')['reading_score'].mean()


# percent passing math
passingScore = 70

cond = (school_data_complete['math_score'] >= passingScore)

schools_summary['% Passing Math'] = school_data_complete[cond].groupby('school_name')['math_score'].count() / school_data_complete.groupby('school_name')['student_name'].count() * 100

# percent passing reading
cond = (school_data_complete['reading_score'] >= passingScore)

schools_summary['% Passing Reading'] = school_data_complete[cond].groupby('school_name')['reading_score'].count() / school_data_complete.groupby('school_name')['student_name'].count() * 100

# percent overall passing rate
schools_summary['% Overall Passing Rate'] = (schools_summary['% Passing Math']+schools_summary['% Passing Reading'])/2


## Top Performing Schools (By Passing Rate)

In [16]:
top_schools = schools_summary.sort_values('% Overall Passing Rate', ascending=False)

top_schools['Total School Budget'] = schools_summary['Total School Budget'].map("${:,.2f}".format)
top_schools['Per Student Budget'] = schools_summary['Per Student Budget'].map("${:,.2f}".format)

top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

In [17]:
bottom_schools = schools_summary.sort_values('% Overall Passing Rate', ascending=True)

bottom_schools['Total School Budget'] = schools_summary['Total School Budget'].map("${:,.2f}".format)
bottom_schools['Per Student Budget'] = schools_summary['Per Student Budget'].map("${:,.2f}".format)

bottom_schools.head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


In [18]:
# Create a table that lists the average Reading Score 
# for students of each grade level (9th, 10th, 11th, 12th)
# at each school.
grades = sorted(list(school_data_complete['grade'].unique()),key=lambda x: int( x.split('th')[0] ))

## Math Scores by Grade

In [19]:
math_by_grade = pd.DataFrame(index=list_of_schools)

for grade in grades:
    math_by_grade[grade] = school_data_complete[school_data_complete['grade'] == grade].groupby('school_name')['math_score'].mean()

math_by_grade = math_by_grade.sort_index()

math_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade

In [20]:
read_by_grade = pd.DataFrame(index=list_of_schools)

for grade in grades:
    read_by_grade[grade] = school_data_complete[school_data_complete['grade'] == grade].groupby('school_name')['reading_score'].mean()

read_by_grade = read_by_grade.sort_index()

read_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending
* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [21]:
# Create a table that breaks down school performances based on
# average Spending Ranges (Per Student). Use 4 reasonable bins 
# to group school spending.
cols = [
    'Average Math Score',
    'Average Reading Score',
    '% Passing Math',
    '% Passing Reading',
    '% Overall Passing Rate'
]

schools_spending = pd.DataFrame()

spending_bins = [0, 585, 615, 645, 675]
group_names = ['<$585', '$585-615', '$615-645', '$645-675']

In [23]:
# Place the data series into a new column inside of the DataFrame
labels = [
    'School Type',
    'Total Students',
    'Total School Budget',
    'Per Student Budget',
    'Average Math Score',
    'Average Reading Score',
    '% Passing Math',
    '% Passing Reading',
    '% Overall Passing Rate'
]

by_school = pd.DataFrame()

spending_bins = [0, 585, 615, 645, 675]
group_names = ['<$585', '$585-615', '$615-645', '$645-675']



## Scores by School Size
* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Scores by School Type
* Perform the same operations as above, based on school type.